In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, GenerationConfig
import torch
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory,ConversationSummaryMemory, ConversationBufferWindowMemory
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser, CommaSeparatedListOutputParser

In [2]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
pipe = pipeline("text-generation", 
                model=model, 
                tokenizer=tokenizer,
                device=5, 
                max_new_tokens=2000,
                do_sample=True, 
                top_k=20, 
                top_p=0.7,
                early_stopping=True,
                num_beams=2
               )
hf = HuggingFacePipeline(pipeline=pipe)

In [4]:
def initialize_response_schemas():
    """Initialises response schemas for StructuredOutputParser"""
    answer_schema = ResponseSchema(name='Answer', description="Correct Answer to the given Question")
    explanation_schema = ResponseSchema(name='Explanation', description = 'Explanation why a particular choice is selected as the answer')
    
    response_schemas = [
                        answer_schema,
                        explanation_schema]
    return response_schemas

In [5]:
response_schemas = initialize_response_schemas()
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Answer": string  // Correct Answer to the given Question
	"Explanation": string  // Explanation why a particular choice is selected as the answer
}
```


In [11]:
template = """Role: You are an expert grader. You can evaluate the Quizzes' answers. Given MCQ Quiz Question along with the answer,
you need to check the answer and provide the correct choice of answer if it is wrong.

User: Below is the input MCQ question with 4 choices and a user-given answer with Explanation. 
Evalaute the answer and give correct choice if answer is wrong. You should not give your chain of though in the response.
Just give the final correct choice of answer along with a brief explanation.

Input: 
Question: {Question} 
Choice1 : {Choice1}
Choice2 : {Choice2}
Choice3 : {Choice3}
Choice4 : {Choice4}
Answer  : {Answer}
Explanation : {Explanation}

{format_instructions}
"""

In [12]:
template2 = """Role: This is different Template for example

User: Below is the input MCQ question with 4 choices and a user-given answer with Explanation. 
Evalaute the answer and give correct choice if answer is wrong. You should not give your chain of though in the response.
Just give the final correct choice of answer along with a brief explanation.

Input: 
Question: {Question} 
Choice1 : {Choice1}
Choice2 : {Choice2}
Choice3 : {Choice3}
Choice4 : {Choice4}
Answer  : {Answer}
Explanation : {Explanation}

{format_instructions}
"""

In [14]:
prompt = PromptTemplate.from_template(template)
prompt2 = PromptTemplate.from_template(template2)
# prompt = PromptTemplate(
#     input_variables=["Question", "Choice1", "Choice2", "Choice3","Choice4","Answer","Explanation",
#                      "format_instructions"
#                     ], template=template
# )

In [59]:
def load_quiz(quiz_path):
    with open(quiz_path,'r') as f:
        data = f.read()

    data = data.split('\n\n')


    ans_list = []
    for ques in data[1:]:
        try:
            _ = ques.split('\n')
            question = _[0].split(':')[1]
            choice1 = _[1].split(':')[1]
            choice2 = _[2].split(':')[1]
            choice3 = _[3].split(':')[1]
            choice4 = _[4].split(':')[1]
            answer = _[5].split(':')[1]
            exp = _[6].split(':')[1]
    
            ques_data = {
                "Question" : question,
                "Choice1" : choice1,
                "Choice2" : choice2,
                "Choice3" : choice3,
                "Choice4" : choice4,
                "Answer" : answer,
                "Explanation" : exp
            }
            ans_list.append(ques_data)
        except Exception as e:
            continue
    return ans_list

In [78]:
quiz_data = load_quiz('./quizzes/quiz_12.txt')
quiz_data[1]

{'Question': 'In an A/B test, two versions of a website are compared to determine which one converts better. Version A has a conversion rate of 12% and a bounce rate of 40%. Version B has a conversion rate of 15% and a bounce rate of 35%. Based on these statistics, which version of the website is more likely to have a lower average session duration?',
 'Choice1': 'Version A',
 'Choice2': 'Version B',
 'Choice3': 'Both versions have the same average session duration',
 'Choice4': 'Insufficient data to determine',
 'Answer': 'Choice3',
 'Explanation': 'Choice3'}

In [8]:
llm_chain = LLMChain(
                    llm=hf,
                    prompt=prompt, 
                    output_parser = output_parser
                )

In [15]:
llm_chain2 = LLMChain(
                    llm=hf,
                    prompt=prompt2, 
                    output_parser = output_parser
                )

In [75]:
data = quiz_data[1]
data['format_instructions'] = format_instructions

In [76]:
out = llm_chain.invoke(data).get('text')

/home/aagarwal/miniconda3/envs/OpenShape/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [77]:
print(out)

{'Answer': 'Choice3', 'Explanation': 'The question does not provide enough data to determine the average session duration for both versions.'}


In [79]:
out = llm_chain.invoke(data).get('text')
print(out)

/home/aagarwal/miniconda3/envs/OpenShape/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'Answer': 'Choice3', 'Explanation': 'Both versions have the same explanation here'}


In [80]:
out = llm_chain.invoke(data).get('text')
print(out)

/home/aagarwal/miniconda3/envs/OpenShape/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'Answer': 'Choice3', 'Explanation': 'Both versions have the same explanation needed for this question'}


In [81]:
out = llm_chain.invoke(data).get('text')
print(out)

/home/aagarwal/miniconda3/envs/OpenShape/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'Answer': 'Choice3', 'Explanation': 'Both versions have the same explanation here'}
